All data reading from database (Mongodb) and crate csv files every individual process

2.The minimum duration of a specific process (e.g. lifting) which gives the minimum possible process time and potential of the process (and the system)

Filter:<br>
01.topic : process <br>
02.element type : Unit procedure<br>
03.cammand key :<br>
'Lift Products'<br>
'Measure products'<br>
'Position Products'<br>
'Punch Products'<br> 
'Glue Products'<br>
'Unlift Products'<br>
'Discharge Row'<br>


In [3]:
#pip install bson
#pip install pymongo

In [4]:
import pandas as pd
import json
import bson

In [5]:
import pymongo
from pymongo import MongoClient
client=MongoClient('localhost' ,27017)

In [6]:
print(client.list_database_names())

['BRIX', 'BRIX1', 'BRIX2', 'admin', 'config', 'ella', 'ella1', 'ella2', 'local']


In [7]:
dblist = client.list_database_names()
if "BRIX2" in dblist:
  print("The database exists.")

The database exists.


In [8]:
mydb = client["BRIX2"]
mydb

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'BRIX2')

In [9]:
print(mydb.list_collection_names())

['events', 'PackML', 'Procedures']


In [10]:
Proceduresdb = mydb["Procedures"]
Proceduresdb 

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'BRIX2'), 'Procedures')

In [11]:
x = Proceduresdb.find_one()
x

{'_id': ObjectId('636bbfb8b157491254152b64'),
 'topic': 'brix/Procedures/Startup/2022-11-09-14:58:15.407_35/Discharge Start row',
 'TimeStamp': datetime.datetime(2022, 11, 9, 15, 10, 27, 125000),
 'contextId': '2022-11-09-14:58:15.407_35',
 'payload': {'commandState': 'execute',
  'stateChangeTime': '2022-11-09T15:10:27.125Z',
  'commandKey': 'Discharge Start row',
  'caller': 'Row discharger',
  'elementType': 'Unit Procedure',
  'result': 0,
  'duration': 0,
  'contextId': '2022-11-09-14:58:15.407_35',
  'executer': ''}}

In [12]:
x['payload']

{'commandState': 'execute',
 'stateChangeTime': '2022-11-09T15:10:27.125Z',
 'commandKey': 'Discharge Start row',
 'caller': 'Row discharger',
 'elementType': 'Unit Procedure',
 'result': 0,
 'duration': 0,
 'contextId': '2022-11-09-14:58:15.407_35',
 'executer': ''}

elementType ['Unit Procedure'] <br>
commandState ['Aborted', 'Completed', 'Stopped', 'execute'] <br>
commandKey ['Discharge Row', 'Glue Products', 'Lift Products', 'Measure products', 'Position Products', 'Punch Products', 'Unlift Products'] <br>
caller ['Product gluer', 'Product lifter', 'Product measurer', 'Product positioner', 'Product puncher', 'Product unlifter', 'Row discharger'] <br>

In [15]:
import datetime
# only available 2022, 11, 9 to 2022, 11, 17
#{ results: { $elemMatch: { $gte: 80, $lt: 85 } } }
start = datetime.datetime(2022, 11, 15)
end = datetime.datetime(2022, 11, 17)
#d = datetime.datetime(2022, 11, 16, 0)
a1=Proceduresdb.find({"TimeStamp": { "$gt": start, "$lte": end }})

In [16]:
df = pd.read_csv('ProcessEgg_UnitProcedure.csv')

dataframes_list = []
dataframes_list1 = []
dataframes_list11 = []
dataframes_list2 = []
dataframes_list3 = []

for va in df['Unit procedure']:
    #filter Process Egg_Unit Procedure_Lift Products
    a2=Proceduresdb.find({"$text": {"$search": "\"Process eggs\""},
                      "payload.elementType": "Unit Procedure",
                      "payload.commandKey": va  })
    
    list_cur2 = list(a2)
    filter_ProcessEgg_UnitProcedure_LiftProducts_df = pd.DataFrame(list_cur2)
    
    #payload extract
    payload=filter_ProcessEgg_UnitProcedure_LiftProducts_df.payload.apply(pd.Series)
    #drop contextId
    payload=payload.drop(['contextId'], axis=1)
    #connect both Data together 
    concatenated = pd.concat([filter_ProcessEgg_UnitProcedure_LiftProducts_df, payload], axis="columns")
    #add date
    concatenated['date'] = [d.date() for d in concatenated['TimeStamp']]
    
    #original data-#process Eggg_unit procedure
    dataframes_list.append(concatenated)
    
    #filter ['commandState']=="Completed",Aborted,Stopped
    b=(concatenated['commandState']=="Completed") | (concatenated['commandState']=="Aborted") | (concatenated['commandState']=="Stopped")
    CAS_df=concatenated[b]
    CAS_df = CAS_df.reset_index()
    CAS_df=CAS_df.drop(['index'], axis=1)
    
    #filter ['commandState']=="execute"
    b1=(concatenated['commandState']=="execute") 
    E_df=concatenated[b1]
    
    # colomn name change
    E_df1=E_df.rename({'TimeStamp':'TimeStamp_start' , 'commandState':'commandState_e', 'result':'result_e'}, axis=1)
    E_df1 = E_df1.reset_index()
    E_df1=E_df1.drop(['index'], axis=1)
    
    #connect both Data together 
    ProcessEgg_df = pd.concat([CAS_df, E_df1['TimeStamp_start'],E_df1['commandState_e'],E_df1['result_e']], axis="columns")
    #remove column
    ProcessEgg_df=ProcessEgg_df.drop(columns=['_id', 'payload', 'stateChangeTime'])

    # append datasets into the list
    #process Eggg_unit procedure
    dataframes_list1.append(ProcessEgg_df)
    
    ##################################### new one 2023.01.02
    #Completed",Aborted,Stopped - process_handling_cycle time calculation
    #Executecompletted_df=ProcessEgg_df[ProcessEgg_df['commandState']=="Completed"]
    E_A_S_df= ProcessEgg_df.copy()
    #calculate process time
    E_A_S_df['process time'] = E_A_S_df['TimeStamp']-E_A_S_df['TimeStamp_start']
    #calculate process time in seconds (execute completed )
    E_A_S_df['process time_s'] = (E_A_S_df['process time']).dt.total_seconds()

    #calculate handling time for execute = execute,Aborted,Stopped time - completed time(TimeStamp column 1 cell up data)
    E_A_S_df1= E_A_S_df.copy()
    E_A_S_df1['handling time'] = E_A_S_df1['TimeStamp_start'].shift(1) - E_A_S_df1['TimeStamp']
    #calculate handling time in seconds 
    E_A_S_df1['handling time_s'] = (E_A_S_df1['handling time']).dt.total_seconds()
    #cycle time LiftProducts (execute to next execute state)
    E_A_S_df1['cycle time_s'] = E_A_S_df1['process time_s'] + E_A_S_df1['handling time_s']
    
    #LiftProducts_Execute_process_handling_cycle time for execute,Aborted,Stopped
    dataframes_list11.append(E_A_S_df1)
    
    #####################################
    
    ####################################
    #filter ['commandState']=="Completed"
    Executecompletted_df=ProcessEgg_df[ProcessEgg_df['commandState']=="Completed"]
    #calculate process time
    Executecompletted_df['process time'] = Executecompletted_df['TimeStamp']-Executecompletted_df['TimeStamp_start']
    #calculate process time in seconds (execute completed )
    Executecompletted_df['process time_s'] = (Executecompletted_df['process time']).dt.total_seconds()

    #calculate handling time for execute = execute time - completed time(TimeStamp column 1 cell up data)
    Executecompletted_df1=Executecompletted_df.copy()
    Executecompletted_df1['handling time'] = Executecompletted_df1['TimeStamp_start'].shift(1) - Executecompletted_df1['TimeStamp']
    #calculate handling time in seconds 
    Executecompletted_df1['handling time_s'] = (Executecompletted_df1['handling time']).dt.total_seconds()
    #cycle time LiftProducts (execute to next execute state)
    Executecompletted_df1['cycle time_s'] = Executecompletted_df1['process time_s'] + Executecompletted_df1['handling time_s']
    
    #LiftProducts_Execute_process_handling_cycle time
    dataframes_list2.append(Executecompletted_df1)
    
    ##############################
    #filter ['commandState']=="Aborted"
    Aborted_df=ProcessEgg_df[ProcessEgg_df['commandState']=="Aborted"]
    #calculate process time Aborted
    Aborted_df['process time_Aborted'] = Aborted_df['TimeStamp']-Aborted_df['TimeStamp_start']
    #calculate process time in seconds (Aborted )
    Aborted_df['process time_Aborted_s'] = (Aborted_df['process time_Aborted']).dt.total_seconds()
    #calculate handling time for execute = execute time - completed time(TimeStamp column 1 cell up data)
    Aborted_df1=Aborted_df.copy()
    Aborted_df1['handling time_Aborted'] = Aborted_df1['TimeStamp_start'].shift(1) - Aborted_df1['TimeStamp']
    #calculate handling time in seconds 
    Aborted_df1['handling time_Aborted_s'] = (Aborted_df1['handling time_Aborted']).dt.total_seconds()
    #cycle time LiftProducts (execute to next execute state)
    Aborted_df1['cycle time_Aborted_s'] = Aborted_df1['process time_Aborted_s'] + Aborted_df1['handling time_Aborted_s']


    # LiftProducts_Aborted_process_handling_cycle time
    dataframes_list3.append(Aborted_df1)
  


C:\Temp\ipykernel_11112\3216544435.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Executecompletted_df['process time'] = Executecompletted_df['TimeStamp']-Executecompletted_df['TimeStamp_start']
C:\Temp\ipykernel_11112\3216544435.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Executecompletted_df['process time_s'] = (Executecompletted_df['process time']).dt.total_seconds()
C:\Temp\ipykernel_11112\3216544435.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

C:\Temp\ipykernel_11112\3216544435.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Executecompletted_df['process time'] = Executecompletted_df['TimeStamp']-Executecompletted_df['TimeStamp_start']
C:\Temp\ipykernel_11112\3216544435.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Executecompletted_df['process time_s'] = (Executecompletted_df['process time']).dt.total_seconds()
C:\Temp\ipykernel_11112\3216544435.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a D

In [17]:
dataframes_list1[0].head()

,topic,TimeStamp,contextId,commandState,commandKey,caller,elementType,result,duration,executer,date,TimeStamp_start,commandState_e,result_e
0,brix/Procedures/Process eggs/2022-11-16-12:10:...,2022-11-16 12:11:36.002,2022-11-16-12:10:43.103_19,Completed,Lift Products,Product lifter,Unit Procedure,2147483647,0,,2022-11-16,2022-11-16 12:11:28.592,execute,0
1,brix/Procedures/Process eggs/2022-11-16-12:10:...,2022-11-16 12:11:27.492,2022-11-16-12:10:34.573_18,Completed,Lift Products,Product lifter,Unit Procedure,2147483647,0,,2022-11-16,2022-11-16 12:11:20.065,execute,0
2,brix/Procedures/Process eggs/2022-11-16-12:10:...,2022-11-16 12:11:17.612,2022-11-16-12:10:24.663_17,Completed,Lift Products,Product lifter,Unit Procedure,2147483647,0,,2022-11-16,2022-11-16 12:11:10.202,execute,0
3,brix/Procedures/Process eggs/2022-11-16-12:10:...,2022-11-16 12:11:09.102,2022-11-16-12:10:10.573_16,Completed,Lift Products,Product lifter,Unit Procedure,2147483647,0,,2022-11-16,2022-11-16 12:11:01.792,execute,0
4,brix/Procedures/Process eggs/2022-11-16-12:10:...,2022-11-16 12:10:59.362,2022-11-16-12:10:00.763_14,Completed,Lift Products,Product lifter,Unit Procedure,2147483647,0,,2022-11-16,2022-11-16 12:10:52.023,execute,0


In [18]:
# export a list of df to separate csv files 
#process Eggg_unit procedure
for j, frame in enumerate(dataframes_list):
    frame.to_csv(f"C:/Users/HLQ/Documents/jupyter/EGSS/Procedures/Process Egg Analysis All/UP_data2/UP_csv/UP{j}.csv")
   


In [19]:
# export a list of df to separate csv files 
#process Eggg_unit procedure -separation
for j1, frame in enumerate(dataframes_list1):
    frame.to_csv(f"C:/Users/HLQ/Documents/jupyter/EGSS/Procedures/Process Egg Analysis All/UP_data2/UP_s_csv/UP_s{j1}.csv")

In [20]:
 #LiftProducts_Execute_process_handling_cycle time for execute,Aborted,Stopped
#dataframes_list11.append(E_A_S_df1_df1)
# export a list of df to separate csv files 
for j11, frame in enumerate(dataframes_list11):
    frame.to_csv(f"C:/Users/HLQ/Documents/jupyter/EGSS/Procedures/Process Egg Analysis All/UP_data2/UP_EAS_csv/UP_E_A_S{j11}.csv")

In [21]:
# export a list of df to separate csv files 
#process_handling_cycle time_Execute state
for j2, frame in enumerate(dataframes_list2):
    frame.to_csv(f"C:/Users/HLQ/Documents/jupyter/EGSS/Procedures/Process Egg Analysis All/UP_data2/UP_E_csv/UP_E{j2}.csv")

In [22]:
# export a list of df to separate csv files 
#process_handling_cycle time_Aborted state
for j3, frame in enumerate(dataframes_list3):
    frame.to_csv(f"C:/Users/HLQ/Documents/jupyter/EGSS/Procedures/Process Egg Analysis All/UP_data2/UP_A_csv/UP_A{j3}.csv")